## Biogas plant

You want to plan the two-year supply of raw materials for a biogas power plant. Such a plant produces energy by burning biogas, which is obtained from the bacterial fermentation of organic wastes. 
Specifically, your plant is powered by corn chopping, a residual of agro-industrial operations that you can purchase from 5 local farms. 
The table below shows the quarterly capacity of each farm for the next two years. Quantities are measured in tons.

Farm|T1|T2|T3|T4|T5|T6|T7|T8
:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
1|700|1500|700|0|0|700|1500|0
2|1350|0|450|0|1350|0|450|0
3|0|1500|1500|0|0|1500|1500|0
4|820|1560|820|0|820|1560|820|0
5|0|680|1080|0|0|680|1080|0

Due to crop rotations and corn harvesting periods, farms are unable to supply material in some quarters. Moreover the types of corn chopping provided are different, each coming with its own unitary purchase price, unitary storage cost and percentage of dry matter. The table below shows a summary of these information.

Farm|Purchase price|Storage cost|Dry matter
:-|:-:|:-:|:-:
1|0.20|0.002|15
2|0.18|0.012|28
3|0.19|0.007|35
4|0.21|0.011|37
5|0.23|0.015|42

Your biogas plant must operate by burning a mixture of corn choppings with a dry matter percentage between 20% and 40%. Under these conditions, the yield is 421.6 kWh of energy per ton of burned material. The energy produced by the plant is sold on the market at a price of 0.28 $/kWh. 

Due to state regulations, all biogas plants can produce a maximum of 1950 MWh of energy per quarter. You are allowed to store corn chopping in a silo, whose total capacity is of 500 tons. 

Plan the supply and inventory of your biogas plant with the goal of maximizing your profits (i.e., revenues minus costs).

In [1]:
# WHAT TO DO: 
# 1) Inspect the model and then code the model in mip and deliver the solution.
# It grants 4points for the lab.
# Each model will be looked at.
# Transform the gas in energy. How to acquire the waist near-by farms over a period of two years.
# There are 5 local farms where to acquire the materials. 
# It is presented of a table the capacity of each farm for the next two years. It is divided in trimesters.
# There are pricings for cropping from different farms. Different costs what you can store for the next period.
# We also need a certain percentage of dry material inside the waist to obtain useful gas to produce energy.
# Dry matter percentage is between 20% and 40%.
# Maximum capacity of 1950MWh of energy per tremester. 
# Store corn chopping in a silo, whose total capacity is of 500 tons.
# Goal: Plan the supply & inventory of this supply gas. Maximize the profits.

# This problem can be translated as a min flow problem where the goal is to reduce the amount of costs
# while achieving the biggest transport of 

# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

#from numpy import array  
#from numpy.linalg import norm 

In [2]:
# (TODO)
import mip

# ------------------------------------ Initialization.

# The purchase price. 
pp = [0.2,0.18,0.19,0.21,0.23]

# The storage price.
sp = [0.002,0.012,0.007,0.011,0.015]

# The dry matter.
dry = [0.15,0.28,0.35,0.37,0.42]

# The quantity stored in the silo.
qty_silo = [0,0,0,0,0] # It is initially empty.

n = len(pp)

# Capacity. Maximum we can use per quarter.
B = [[700,1350,0,820,0],
    [1500,0,1500,1560,680],
    [700,450,1500,820,1080],
    [0,0,0,0,0],
    [0,1350,0,820,0],
    [700,0,1500,1560,680],
    [1500,450,1500,820,1080],
    [0,0,0,0,0]]

def solve_furnace(b,pp,sp,dry,qty_silo):
    
    n = len(pp)
    
    # Let's create the model().
    m = mip.Model()
    
    # We then add the variables. 
    x = [m.add_var() for j in range(n)] # qty of crops.
        
    # Let's add the constraints.
    m.add_constr(mip.xsum((dry[j]-0.4)*x[j] for j in range(n)) <= 0) # Respect the 40% rule.
    m.add_constr(mip.xsum((dry[j]-0.2)*x[j] for j in range(n)) >= 0) # Respect the 20% rule.
    m.add_constr(mip.xsum(426.1 * (x[j] + qty_silo[j]) for j in range(n)) <= 1950*10**3) # Respect the production per quarter.
    for i in range(n):
        m.add_constr(x[i] <= b[i])
    
    # Objective function represents the profit (Revenues - Costs).
    # We precise that the costs are composed of --> costs = purchase + storage prices.
    m.objective = mip.maximize(mip.xsum([(0.28*426.1)*x[j] + (0.28*426.1)*qty_silo[j] # Our revenue.
                                         - (pp[j]*x[j]) # For the purchases.
                                         - (sp[j]*(b[j]-x[j])) for j in range(n)])) # For the costs of storage.
    
    m.optimize()
    # Return a tuple containing model, solution and the objective value.
    return (m, [x[j].x for j in range(n)], m.objective_value)

def solve_silo(sp,left_overs):
    
    n = len(sp)
    
    # We suppose that for each quantity stored inside the silo, they will be prioritized for the production of energy.
    # In fact, using the storage from last quarter will prevent from spending more in purchases and we will spend less
    # in storage. However, when it comes to storing crops, not all of them are equivalent f.i. with 7.5 tons of type 1,
    # we can store as much as 1 ton of type 5 (0.002$/ton type 1 vs. 0.015$/ton type 5). 
    
    # We take the smallest price of storage from the list and normalize around it. 
    # This way, we give an order of importance from the cheapest crop to store to the most expensive one.
    norm_sp = [min(sp)/sp[j] for j in range(n)]
    
    # Let's create the model
    m = mip.Model()
    
    # Let's add the variables.
    y = [m.add_var() for j in range(n)] # quantities to be stored inside the silo.
    
    # Let's add the constraints.
    m.add_constr(mip.xsum(y[j] for j in range(n)) <= 500) # Maximum capacity of storage of the silo is 500kg.
    for i in range(n):
        m.add_constr(y[i] <= left_overs[i])
    
    # The objective function represents the fact that we want to store as much as "quality" crops first in terms of 
    # storage costs as we want. (y1>y3>y4>y2>y5) is the order of priority.
    m.objective = mip.maximize(mip.xsum([norm_sp[j] * y[j] for j in range(n)]))
    
    m.optimize()
    
    return (m, [y[j].x for j in range(n)], m.objective_value)

In [3]:
# ----------------------- Treatment. 

# We insert them at the beginning of the treatment part. For 'refreshing' purposes we want them empty
# every time we refresh this cell.

# To keep track of the profits.
profits = {}

# To keep track of the losses 
costs = {}

# To keep track of what's inside the silo every quarter.
storage = {}

for i in range(8):
    # We solve the model the model for each quarter.
    _, solution, objective = solve_furnace(B[i],pp,sp,dry,qty_silo)
    #print("Solution", solution) # indiquer pour quel jour à chaque fois.
    #print("Objective: {0:10.2f}".format(objective)) # Indiquer quel jour à chaque fois aussi. 
    
    string = "Term" + ' ' + str(i+1)

    # Keeping track of the profit by the end of every quarters.    
    profits[string] = objective
        
    # Keeping track of the costs by the end of every quarters. (Purchase + Storage) = Revenue - Profits.
    costs[string] = sum([(0.28*426.1)*solution[j] + (0.28*426.1)*qty_silo[j] for j in range(n)]) - objective
    
    if solution == B[i]: # The crops to use for the furnace have been maxed out.
        print("No need to use the silo for " + string)
        qty_silo = [0,0,0,0,0]
    else :
        # The left_overs (when the furnace is full for the quarter or the reaction can't occur anymore).
        left_overs = [B[i][j] - solution[j] for j in range(n)] # The capacity - what has been used.
        #print(left_overs)
        
        # We run the model for the silo.
        _, available, _ = solve_silo(sp,left_overs)
        qty_silo = available # The crops that have been put inside the silo are available for the next quarter.

    # Keep track of how much is stored by the end of every quarters.
    storage[string] = tuple(qty_silo)

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 28 2021 

Starting solution of the Linear programming problem using Primal Simplex

No need to use the silo for Term 1
Coin0506I Presolve 0 (-8) rows, 0 (-5) columns and 0 (-20) elements
Clp0000I Optimal - objective value 341885.38
Coin0511I After Postsolve, objective 341885.38, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 341885.38 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00
Starting solution of the Linear programming problem using Primal Simplex

Coin0506I Presolve 3 (-5) rows, 4 (-1) columns and 12 (-8) elements
Clp1000I sum of infeasibilities 0 - average 0, 0 fixed columns
Coin0506I Presolve 3 (0) rows, 4 (0) columns and 12 (0) elements
Clp0029I End of values pass after 4 iterations
Clp0000I Optimal - objective value 545114.54
Clp0000I Optimal - objective value 545114.54
Clp0000I Optimal - objective value 545114.54
Coin0511I After Postsolve, objective 545114.54, infeasibilities - dual 0

In [4]:
# ----------------------- Display.
print(profits)
print(costs)
print(storage)

{'Term 1': 341858.76000000007, 'Term 2': 545073.6760384886, 'Term 3': 545175.2260384888, 'Term 4': 56505.39999999996, 'Term 5': 258483.16000000006, 'Term 6': 528818.5200000001, 'Term 7': 545079.7260384887, 'Term 8': 59654.00000000001}
{'Term 1': 555.1999999999534, 'Term 2': 926.3239615114871, 'Term 3': 824.7739615113242, 'Term 4': 0.0, 'Term 5': 415.19999999998254, 'Term 6': 909.0, 'Term 7': 920.2739615114406, 'Term 8': 0.0}
{'Term 1': (0, 0, 0, 0, 0), 'Term 2': (0.0, 0.0, 0.0, 0.0, 500.0), 'Term 3': (0.0, 0.0, 0.0, 0.0, 473.60948134240743), 'Term 4': (0, 0, 0, 0, 0), 'Term 5': (0, 0, 0, 0, 0), 'Term 6': (0, 0, 0, 0, 0), 'Term 7': (0.0, 0.0, 0.0, 0.0, 500.0), 'Term 8': (0, 0, 0, 0, 0)}
